## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray Resources/establishments.json`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
!mongoimport --type json -d uk_food -c establishments --drop --jsonArray Resources/establishments.json

2025-01-02T14:23:48.555-0600	connected to: mongodb://localhost/
2025-01-02T14:23:48.555-0600	dropping: uk_food.establishments
2025-01-02T14:23:49.808-0600	39779 document(s) imported successfully. 0 document(s) failed to import.


In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'epa_real', 'local', 'met', 'petsitly_marketing', 'test', 'travel_db', 'uk_food']


In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [7]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': '1 The Droveway',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'FirstLight Bar & Café',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.64041272672,
 'FHRSID': 1290215,
 'LocalAuthorityBusinessID': 'PI/000083029',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DH',
 'RatingDate': '2020-11-09T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6776f5d4e648226b03f52372'),
 'geocode': {'latitude': '51.150625', 'longitude': '1.378231'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1290215',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '

In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [10]:
# Create a dictionary for the new restaurant data and insert into the collection
establishments.insert_one(
    {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
})

InsertOneResult(ObjectId('6776f614a260d3a0553f5dd9'), acknowledged=True)

In [11]:
# Check that the new restaurant was inserted
query = {"BusinessName": "Penang Flavours"}
results = establishments.find(query)
for result in results:
    pprint(result)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6776f614a260d3a0553f5dd9'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [17]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': {'$regex': 'Restaurant/Cafe/Canteen'}}

fields = {'BusinessTypeID': 1, 'BusinessType':1}

pprint(list(establishments.find(query, fields).limit(1))) #why is '_id' showing up?

[{'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('6776f5d4e648226b03f52372')}]


3. Update the new restaurant with the `BusinessTypeID` you found.

In [21]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    {'BusinessName': 'Penang Flavours'},
    {'$set':
         {'BusinessTypeID': 1}
    } 
 )

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [22]:
# Confirm that the new restaurant was updated
query = {"BusinessName": "Penang Flavours"}
results = establishments.find(query)
for result in results:
    pprint(result)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6776f614a260d3a0553f5dd9'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [24]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': 'Dover'}

print(establishments.count_documents(query))

994


In [25]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {'LocalAuthorityName': 'Dover'}

establishments.delete_many(query)

DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [26]:
# Check if any remaining documents include Dover
query = {'LocalAuthorityName': 'Dover'}

print(establishments.count_documents(query))

0


In [27]:
# Check that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('6776f5d4e648226b03f52657'),
 'FHRSID': 289352,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000002460',
 'BusinessName': 'The Ship',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'AddressLine1': '23 The Stade',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT19 6AB',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2015-08-27T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.188537', 'latitude': '51.08084'},
 'RightToReply': '',
 'Distance': 4591.912144538092,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount': 0,

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [33]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [
    {'$set': {
        "geocode.longitude": {'$toDecimal': "$geocode.longitude"},
        "geocode.latitude": {'$toDecimal': "$geocode.latitude"}
    }}
])

UpdateResult({'n': 38786, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [34]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [37]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [
    {'$set': {
        "RatingValue": {'$toInt': '$RatingValue'}
    }}
])

UpdateResult({'n': 38786, 'nModified': 34695, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [38]:
# Check that the coordinates and rating value are now numbers
establishments.find_one() #cannot for the life of me remember how to look at types in mongo

{'_id': ObjectId('6776f5d4e648226b03f52657'),
 'FHRSID': 289352,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000002460',
 'BusinessName': 'The Ship',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'AddressLine1': '23 The Stade',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT19 6AB',
 'Phone': '',
 'RatingValue': 5,
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2015-08-27T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': Decimal128('1.18853700000000'),
  'latitude': Decimal128('51.0808400000000')},
 'RightToReply': '',
 'Distance': 4591.912144538092,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': 